# About This Notebook
This Notebook is an exploration notebook to learn GBXML data manipulation created by [Martin M.](https://www.mctm.web.id)

# Prerequisites
- Before you start, as written on the [documentation](https://xgbxml.readthedocs.io/en/latest/installation.html) you have to install some basic packages with the command below:
    
    ```bash
    pip install lxml xgbxml shapely triangle matplotlib
    ```

- I strongly recommend you to create a new python environment using conda or virtualenv to avoid any package conflict.
- Also, if you run this notebook using vscode, you need to install `ipykernel` package to run this notebook with this command:

    ```bash
    pip install ipykernel
    ```
---

# How to Open GBXML File

## First Method: Using LXML

In [1]:
from lxml import etree
tree=etree.parse('gbXMLStandard.xml')
gbxml=tree.getroot()
print(type(gbxml))
print(len(list(gbxml)))
print(gbxml.attrib)

<class 'lxml.etree._Element'>
2405
{'useSIUnitsForResults': 'true', 'temperatureUnit': 'C', 'lengthUnit': 'Meters', 'areaUnit': 'SquareMeters', 'volumeUnit': 'CubicMeters', 'version': '0.37'}


## Second Method: Using XGBXML

In [2]:
from lxml import etree
import xgbxml

parser=xgbxml.get_parser('0.37')   
tree=etree.parse('gbXMLStandard.xml', parser)
gbxml=tree.getroot()
print(type(gbxml))        
print(len(list(gbxml)))   

<class 'xgbxml.xgbxml.gbXML'>
2405


---

# Query a GBXML File

## Using LXML

In [3]:
from lxml import etree

tree=etree.parse('gbXMLStandard.xml')
gbxml=tree.getroot()
print(gbxml.attrib['version'])  
print(gbxml[0].tag)
print(gbxml[0][0][0].text)

0.37
{http://www.gbxml.org/schema}Campus
650982_2006


## Using XGBXML

In [4]:
tree=etree.parse('gbXMLStandard.xml')
gbxml=tree.getroot()

ns={'gbxml':'http://www.gbxml.org/schema'}
print(gbxml.xpath('./@version')[0])  
print(gbxml.xpath("./gbxml:*", namespaces=ns)[0].tag)
print(gbxml.xpath(".//gbxml:StationId/text()", namespaces=ns)[0])
print(len(gbxml.xpath(".//gbxml:*", namespaces=ns)))

0.37
{http://www.gbxml.org/schema}Campus
650982_2006
294708


# Query Example

## Areas by Surface Type

In [5]:
from lxml import etree
import xgbxml
from pprint import pprint
parser = xgbxml.get_parser('0.37')
tree = etree.parse('gbXMLStandard.xml', parser)
gbxml = tree.getroot()

result = {}
for surface in gbxml.Campus.Surfaces:
    surface_type = surface.surfaceType
    surface_plus_openings_area = surface.PlanarGeometry.get_area()
    x = result.setdefault(surface_type,0)
    result[surface_type] = x + surface_plus_openings_area

pprint(result)


{'Ceiling': 76.97475404196999,
 'ExteriorWall': 5563.731124015909,
 'InteriorFloor': 6359.7906685989465,
 'InteriorWall': 5115.973792739992,
 'RaisedFloor': 168.78740766564601,
 'Roof': 423.8474802081981,
 'Shade': 283.91268931643106,
 'SlabOnGrade': 243.2793321101385}


## Areas and window-to-wall ratio by orientation

In [6]:
from lxml import etree
import xgbxml
from pprint import pprint

parser=xgbxml.get_parser('0.37')
tree=etree.parse('gbXMLStandard.xml', parser)  # file available on GitHub here: https://github.com/GreenBuildingXML/Sample_gbXML_Files
gbxml=tree.getroot()
result={}
for surface in gbxml.Campus.Surfaces:

    cad_model_azimuth = gbxml.Campus.Location.CADModelAzimuth.value
    surface_azimuth = surface.RectangularGeometry.Azimuth.value
    orientation = cad_model_azimuth + surface_azimuth

    total_area=surface.PlanarGeometry.get_area()
    surface_area=surface.get_area()

    opening_area = total_area - surface_area

    x=result.setdefault(orientation,
                        {'opening_area':0,
                         'surface_area':0,
                         'total_area':0}
                       )
    x['opening_area']+=opening_area
    x['surface_area']+=surface_area
    x['total_area']+=total_area

for k,v in result.items():
    result[k]['window_to_wall_ratio']=result[k]['opening_area'] / result[k]['total_area']

result = dict(sorted(result.items()))  # sorts the dictionary by key

pprint(result)

{57.0: {'opening_area': 581.2910685181002,
        'surface_area': 4437.984049038669,
        'total_area': 5019.275117556765,
        'window_to_wall_ratio': 0.11581175665880926},
 101.77: {'opening_area': 0.0,
          'surface_area': 1.379421701942,
          'total_area': 1.379421701942,
          'window_to_wall_ratio': 0.0},
 147.0: {'opening_area': 0.0,
         'surface_area': 1678.383655294186,
         'total_area': 1678.383655294186,
         'window_to_wall_ratio': 0.0},
 237.0: {'opening_area': 666.6709195756767,
         'surface_area': 3911.492811425113,
         'total_area': 4578.163731000785,
         'window_to_wall_ratio': 0.14561971977134655},
 326.87: {'opening_area': 0.0,
          'surface_area': 4.068926225344818,
          'total_area': 4.068926225344818,
          'window_to_wall_ratio': 0.0},
 327.0: {'opening_area': 57.16800044196051,
         'surface_area': 6895.968675648382,
         'total_area': 6953.136676090338,
         'window_to_wall_ratio': 0.00

In [ ]:
for key in gbxml.Campus[1]:
    print(key)

In [10]:
from gbxmlhelper import find_thekey
list_keys, list_num_keys = find_thekey(gbxml.Campus.Building)

print(f"list_keys: {list_keys}")
print(f"list_num_keys: {list_num_keys}")

list_keys: ['StreetAddress', 'Area', 'BuildingStorey', 'Space', 'Name', 'Description', 'Lighting', 'PeakDomesticHotWaterFlow']
list_num_keys: [1, 1, 20, 431, 2, 2, 1, 1]


In [ ]:
for idx, key in enumerate(gbxml.Campus):
    key_tag = key.tag.replace('{http://www.gbxml.org/schema}', '')
    # print(f'{idx}: {key_tag}')
    if key_tag == 'Building':
        print(f'{idx}: {key_tag}')

In [ ]:
from gbxmlhelper import search_its_idx

idx = search_its_idx(gbxml.Campus, 'Building')
print(f"Building is at index {idx}")